In [1]:
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
%matplotlib inline
import gc
gc.collect()

4

In [52]:
X = np.array([[0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0],
                                [1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0],
                                [2, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 1],
                                [1, 1, 0, 1, 1, 1], [1, 1, 0, 0, 1, 0],
                                [1, 1, 1, 1, 1, 0], [0, 2, 2, 0, 2, 1],
                                [2, 2, 2, 2, 2, 0], [2, 0, 0, 2, 2, 1],
                                [0, 1, 0, 1, 0, 0], [2, 1, 1, 1, 0, 0],
                                [1, 1, 0, 0, 1, 1], [2, 0, 0, 2, 2, 0],
                                [0, 0, 1, 1, 1, 0]])
y = np.array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

X_test=np.array([[0, 0, 1, 0, 0, 0], [1, 0, 1, 0, 0, 0],
                    [1, 1, 0, 1, 1, 0], [1, 0, 1, 1, 1, 0],
                     [1, 1, 0, 0, 1, 1], [2, 0, 0, 2, 2, 0],
                     [0, 0, 1, 1, 1, 0],
                     [2, 0, 0, 2, 2, 0],
                     [0, 0, 1, 1, 1, 0]
                     ])

In [85]:
class NBclassifier:
    def __init__(self, n_class=2, smooth=False):
        self.pxc = None
        self.n_class = n_class
        self.pc = None
        self.smooth = smooth #是否平滑
        
    def fit(self, X, y):
        n_attr = X.shape[1] 
        y_unique = np.sort(np.unique(y)) #并从小到大排列独立值
        self.pc = np.zeros(len(y_unique))
        if self.smooth:
            for i in range(len(y_unique)):
                self.pc[i] = len(np.where(y==y_unique[i])[0] + 1) / (len(y) + len(y_unique)) #公式7.19
        else:
            for i in range(len(y_unique)):
                self.pc[i] = len(np.where(y==y_unique[i])[0]) / len(y) #公式7.16

        #对于离散属性 
        pxc = {}
        for c in y_unique:
            Dcx = {}
            indices = np.where(y==c)[0]
            for attr_index in range(n_attr): #对于每一个特征
                pxic = {}
                attr_unique = np.unique(X[indices][:, attr_index])  #该字段有多少种x值
                for xi in attr_unique:
                    dvc = np.where(X[indices][:, attr_index]==xi)[0]
                    if self.smooth:
                        pxic[xi] = (len(dvc) +1) / (len(indices)+len(attr_unique)) #公式7.20
                    else:
                        pxic[xi] = len(dvc) / len(indices) #公式7.17
                    
                Dcx[attr_index] = pxic
            pxc[c] = Dcx
        self.pxc = pxc

    def predict(self, X):
        p = []
        for x in range(X.shape[0]):
            pmax = 0
            y_hat = None
            for c in range(self.n_class):
                p_0 = self.pc[c]
                Dcx = self.pxc[c]
                for attr_index, pxic in Dcx.items():
                    xi = X[x][attr_index] #当前sample的xi特征值
                    if xi in pxic.keys():  #可能存在训练集在该类别中，该值未出现
                        p_0 *= pxic[xi]
                    else:
                        if self.smooth:
                            p_0 *= 1 / len(pxic.keys())
                        else:
                            p_0 *= 0
                        
                if p_0 >= pmax:
                    pmax = p_0
                    y_hat = c
            p.append(y_hat)
            
        return p
                    
nb = NBclassifier()
nb.fit(X, y)
y_hat = nb.predict(X_test)
print(y_hat)
        

[1, 1, 1, 1, 1, 0, 0, 0, 0]
